# **마을화폐 광 Crawling**
## **1 Web Crawling**
웹사이트 데이터 수집 및 전처리 작업

In [1]:
# Crawling the Sites
# http://o2o.maulo.kr/list/account/tradinglst/?pageNo=106&tradings=9
from urllib import parse, request

def kwangCoin(page:int):
    r"""마을화폐 광 Crawling to Excel
    http://o2o.maulo.kr/list/account/tradinglst/?pageNo=106&tradings=9"""
    query = {
        "pageNo":page, # 106,
        "qrSt":"tradinglst",
        "hom_cd":"100000106",
        "tradings":9
    }
    url_query = parse.urlencode(query, encoding='utf-8', doseq=True)
    url_head = "http://o2o.maulo.kr/list/account/tradinglst/"
    browser = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/534.55.3 (KHTML, like Gecko) Version/5.1.5 Safari/534.55.3"
    return request.urlopen(request.Request(url_head + "?" + url_query,
        headers = {'User-Agent': browser})).read().decode('utf-8')

In [2]:
# Filtering the Table
import re
import numpy as np
from lxml import html

def extractData(response):
    result_data = []
    tokens = re.findall('<td.*?>(.*?)<td>', response)
    indexMax = np.argmax([len(_) for _ in tokens])
    tokens = tokens[indexMax] # 가장 유효값 높은 자료 필터링
    token_reg = '<tr onMouseOver="style.backgroundColor=(.*?)</tr>'

    # Filtering the Cell Data
    for datum in re.findall(token_reg, tokens):
        datum = datum.replace("\'f4f4f4\'", "")
        datum = datum.replace("onMouseOut=\"style.backgroundColor=\'\'", "")
        datum = datum.replace("\" \">", "")
        html_lxml = html.fromstring(datum)
        result_data.append(html_lxml.xpath("//text()"))
    return result_data

In [5]:
response = kwangCoin(1)
extractData(response)[:5]

[['3176', '김**', '송금', '50,000', '0', '2020-09-22'],
 ['3175', '김**', '송금', '50,000', '0', '2020-09-22'],
 ['3174', '김**', '송금', '100,000', '0', '2020-09-22'],
 ['3173', '김**', '송금', '100,000', '0', '2020-09-22'],
 ['3172', '김**', '송금', '100,000', '0', '2020-09-22']]

## **2 작업내용 CSV 저장하기**
수집 및 전처리 함수의 실행

In [6]:
from tqdm import tqdm
from time import sleep
import pandas as pd
result = []
for num in tqdm(range(1, 107)):
    sleep(0.5)
    response = kwangCoin(num)
    result += extractData(response)
    
result = pd.DataFrame(result)
result.columns = ['index', "buyer", "seller", "price", "saving", "date"]

100%|██████████| 106/106 [04:39<00:00,  2.63s/it]


,0,1,2,3,4,5
0,3176,김**,송금,"50,000",0,2020-09-22
1,3175,김**,송금,"50,000",0,2020-09-22
2,3174,김**,송금,"100,000",0,2020-09-22


In [8]:
result.columns = ['index', "buyer", "seller", "price", "saving", "date"]
result.to_csv("coindata.csv", index=None)
result.head(3)

,index,buyer,seller,price,saving,date
0,3176,김**,송금,"50,000",0,2020-09-22
1,3175,김**,송금,"50,000",0,2020-09-22
2,3174,김**,송금,"100,000",0,2020-09-22
